In [1]:
import os
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
os.chdir(parent_directory)

# Create Ego-sync datasets for BEVDet

We take Waymo as the reference dataset and transform ego frames of other datasets

In [2]:
argo_egoZ = [dict(type='EgoTranslate', trans = [-0.75,0,-0.3488])]
kitti_egoZ = [dict(type='EgoTranslate', trans = [-3.0,0,-1.73])]
K360_egoZ = [dict(type='EgoTranslate', trans = [-3.0,0,-1.73])]
nusc_egoZ = [dict(type='ego_transform'), dict(type='EgoTranslate', trans = [-1.0,0,0])]
lyft_egoZ = [dict(type='ego_transform'), dict(type='EgoTranslate', trans = [-2.0,0,0])]

In [4]:
import pandas
import numpy as np
from mmdet3d.structures import LiDARInstance3DBoxes,CameraInstance3DBoxes
import mmengine
argo = ['data/argo2/argo2_infos_val_2Hz_part_mono_front.pkl','data/argo2/argo2_infos_train_2Hz_mono_front.pkl']
kitti = ['data/kitti/kitti_infos_val.pkl','data/kitti/kitti_infos_train.pkl']
K360 = ['data/kitti-360/kitti360_infos_val.pkl','data/kitti-360/kitti360_infos_train.pkl']
nusc = ['data/nus_v2/nuscenes_infos_val_part_mono_front.pkl','data/nus_v2/nuscenes_infos_train_mono_front.pkl']
lyft = ['data/lyft/lyft_infos_val_mono_front.pkl', 'data/lyft/lyft_infos_train_mono_front.pkl']
pkl = nusc[0]
infos = pandas.read_pickle(pkl)


## Step1: nuScenes and Lyft

In [6]:
# nusc_egoZ = [dict(type='ego_transform'), dict(type='EgoTranslate', trans = [-1.0,0,0])]
# lyft_egoZ = [dict(type='ego_transform'), dict(type='EgoTranslate', trans = [-2.0,0,0])]  
ts = np.array([[-1.0,0,0],[-2.0,0,0]])
for t, datasets in zip(ts,[nusc,lyft]):
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        for info in infos['data_list']:
            # what to change: instances, cam2ego, lidar2cam, ego2global, lidar2ego
            Tr1 = np.array(info['lidar_points']['lidar2ego'])
            Tr2 = np.identity(4)
            Tr2[:3,3] = -t
            Tr = Tr1 @ Tr2
            inv_Tr = np.linalg.inv(Tr)

            bboxes = [inst['bbox_3d'] for inst in info['instances']]
            gt_bbox_3d= LiDARInstance3DBoxes(np.array(bboxes),box_dim=7,origin=(0.5, 0.5, 0.5))
            gt_bbox_3d.rotate(Tr[:3,:3].T)
            gt_bbox_3d.translate(Tr[:3,3])
            xyz, lwh, yaw= gt_bbox_3d.gravity_center.tolist(), gt_bbox_3d.dims.tolist(), gt_bbox_3d.yaw.tolist()
            for i in range(len(info['instances'])):
                info['instances'][i]['bbox_3d'] = [*xyz[i],*lwh[i],yaw[i]]

            e2g = info['ego2global']
            c2e = info['images']['CAM_FRONT']['cam2ego']
            l2c = info['images']['CAM_FRONT']['lidar2cam']
            info['images']['CAM_FRONT']['lidar2cam'] = (np.array(l2c) @ inv_Tr).tolist()
            info['images']['CAM_FRONT']['cam2ego'] = (Tr @ np.array(c2e)).tolist()
            info['ego2global'] = (np.array(e2g) @ inv_Tr).tolist()
        mmengine.dump(infos,ds.replace('.pkl','_egoalign.pkl'))
        print('done with', ds)

done with data/nus_v2/nuscenes_infos_val_part_mono_front.pkl
done with data/nus_v2/nuscenes_infos_train_mono_front.pkl
done with data/lyft/lyft_infos_val_mono_front.pkl
done with data/lyft/lyft_infos_train_mono_front.pkl


## Step2: argoverse2, KITTI360

In [5]:
# argo_egoZ = [dict(type='EgoTranslate', trans = [-0.75,0,-0.3488])]
# K360_egoZ = [dict(type='EgoTranslate', trans = [-3.0,0,-1.73])]
ts = np.array([[-0.75,0,-0.3488],[-3.0,0,-1.73]])

for t, datasets,CAM_FRONT in zip(ts,[argo,K360],['ring_front_center','CAM0']):
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        for info in infos['data_list']:
            # what to change: instances, lidar2cam
            # what ignore: ego2global
            # what does not exist: cam2ego
            Tr1 = np.identity(4)
            Tr2 = np.identity(4)
            Tr2[:3,3] = -t
            Tr = Tr1 @ Tr2
            inv_Tr = np.linalg.inv(Tr)
            # print(info['instances'])
            bboxes = [inst['bbox_3d'] for inst in info['instances']]
            gt_bbox_3d= LiDARInstance3DBoxes(np.array(bboxes),box_dim=7,origin=(0.5, 0.5, 0.5))

            gt_bbox_3d.rotate(Tr[:3,:3].T)
            gt_bbox_3d.translate(Tr[:3,3])
            xyz, lwh, yaw= gt_bbox_3d.gravity_center.tolist(), gt_bbox_3d.dims.tolist(), gt_bbox_3d.yaw.tolist()
            for i in range(len(info['instances'])):
                info['instances'][i]['bbox_3d'] = [*xyz[i],*lwh[i],yaw[i]]
            # print(info['instances'])
            l2c = info['images'][CAM_FRONT]['lidar2cam']
            info['images'][CAM_FRONT]['lidar2cam'] = (np.array(l2c) @ inv_Tr).tolist()
            # break
        mmengine.dump(infos,ds.replace('.pkl','_egoalign.pkl'))
        print('done with', ds)

done with data/argo2/argo2_infos_val_2Hz_part_mono_front.pkl
done with data/argo2/argo2_infos_train_2Hz_mono_front.pkl
done with data/kitti-360/kitti360_infos_val.pkl
done with data/kitti-360/kitti360_infos_train.pkl


## Step3: KITTI

In [13]:
# argo_egoZ = [dict(type='EgoTranslate', trans = [-0.75,0,-0.3488])]
# K360_egoZ = [dict(type='EgoTranslate', trans = [-3.0,0,-1.73])]
ts = np.array([-3.0,0,-1.73])
# from mmdet3d.structures.bbox_3d import box_3d_mode
LIDAR = 0
CAM = 1
DEPTH = 2
for t, datasets,CAM_FRONT in zip([ts],[kitti],['CAM2']):
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        for info in infos['data_list']:
            # what to change: instances, lidar2cam
            # what ignore: ego2global
            # what does not exist: cam2ego
            Tr1 = np.identity(4)
            Tr2 = np.identity(4)
            Tr2[:3,3] = -t
            Tr = Tr1 @ Tr2
            inv_Tr = np.linalg.inv(Tr)
            l2c = info['images'][CAM_FRONT]['lidar2cam']
            # bboxes = [inst['bbox_3d'] for inst in info['instances']]
            # gt_bbox_3d= LiDARInstance3DBoxes(np.array(bboxes),box_dim=7,origin=(0.5, 0.5, 0.5))
            # gt_bbox_3d = CameraInstance3DBoxes(np.array(bboxes))
            # gt_bbox_3d = gt_bbox_3d.convert_to(LIDAR, np.linalg.inv(l2c))
            # gt_bbox_3d.rotate(Tr[:3,:3].T)
            # gt_bbox_3d.translate(Tr[:3,3])
            # gt_bbox_3d = gt_bbox_3d.convert_to(CAM, l2c)
            # xyz, lwh, yaw= gt_bbox_3d.gravity_center.tolist(), gt_bbox_3d.dims.tolist(), gt_bbox_3d.yaw.tolist()
            # for i in range(len(info['instances'])):
            #     info['instances'][i]['bbox_3d'] = [*xyz[i],*lwh[i],yaw[i]]
            info['images'][CAM_FRONT]['lidar2cam'] = (np.array(l2c) @ inv_Tr).tolist()
        mmengine.dump(infos,ds.replace('.pkl','_egoalign.pkl'))
        print('done with', ds)

done with data/kitti/kitti_infos_val.pkl
done with data/kitti/kitti_infos_train.pkl


## Sanity check

In [ ]:
print(infos['data_list'][0].keys())
print(infos['data_list'][0]['images']['CAM_FRONT'].keys())
print(infos['data_list'][0]['ego2global'])
print(infos['data_list'][0]['images']['CAM_FRONT']['cam2ego'])
print(infos['data_list'][0]['images']['CAM_FRONT']['lidar2cam'])
print(infos['data_list'][0]['instances'][0])
print(infos['data_list'][1]['token'])


In [ ]:
for datasets in [argo,kitti,K360,nusc,lyft]:
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        print(infos['data_list'][0].keys())
        break

dict_keys(['sample_idx', 'log_id', 'city_name', 'timestamp', 'ego2global', 'images', 'lidar_points', 'instances'])
dict_keys(['sample_idx', 'images', 'lidar_points', 'instances', 'cam_instances'])
dict_keys(['sample_idx', 'log_id', 'timestamp', 'cam0_to_global', 'images', 'lidar_points', 'instances'])
dict_keys(['sample_idx', 'token', 'timestamp', 'ego2global', 'images', 'lidar_points', 'instances', 'city_name'])
dict_keys(['sample_idx', 'token', 'timestamp', 'ego2global', 'images', 'lidar_points', 'lidar_sweeps', 'instances', 'city_name'])


In [ ]:
for datasets in [argo,kitti,K360,nusc,lyft]:
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        print(infos['data_list'][0]['lidar_points'].keys())
        break

dict_keys(['num_pts_feats', 'lidar_path', 'lidar2ego_down', 'lidar2ego_up', 'timestamp'])
dict_keys(['num_pts_feats', 'lidar_path', 'Tr_velo_to_cam', 'Tr_imu_to_velo'])
dict_keys(['num_pts_feats', 'lidar_path', 'lidar2ego', 'timestamp'])
dict_keys(['num_pts_feats', 'lidar_path', 'lidar2ego'])
dict_keys(['num_pts_feats', 'lidar_path', 'lidar2ego'])


In [ ]:
for datasets in [argo,kitti,K360,nusc,lyft]:
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        print(infos['data_list'][0]['images'].keys())
        break

dict_keys(['ring_front_center'])
dict_keys(['CAM0', 'CAM1', 'CAM2', 'CAM3', 'R0_rect'])
dict_keys(['CAM0', 'CAM1'])
dict_keys(['CAM_FRONT'])
dict_keys(['CAM_FRONT'])


In [ ]:
for i,datasets in zip(['ring_front_center','CAM2','CAM0','CAM_FRONT','CAM_FRONT'],[argo,kitti,K360,nusc,lyft]):
    for ds in datasets:
        infos = pandas.read_pickle(ds)
        print(infos['data_list'][0]['images'][i].keys())
        break

dict_keys(['img_path', 'height', 'width', 'cam2img', 'lidar2cam', 'timestamp'])
dict_keys(['img_path', 'height', 'width', 'cam2img', 'lidar2img', 'lidar2cam'])
dict_keys(['img_path', 'height', 'width', 'cam2img', 'lidar2cam', 'timestamp'])
dict_keys(['img_path', 'cam2img', 'cam2ego', 'sample_data_token', 'timestamp', 'lidar2cam'])
dict_keys(['img_path', 'cam2img', 'cam2ego', 'sample_data_token', 'timestamp', 'lidar2cam'])


In [ ]:
import numpy as np
from mmdet3d.structures import LiDARInstance3DBoxes,CameraInstance3DBoxes
import mmengine
inst = infos['data_list'][1]['instances'][0]
gt_bbox_3d = np.array(inst['bbox_3d'])
print(gt_bbox_3d)
gt_bbox_3d= LiDARInstance3DBoxes([gt_bbox_3d],box_dim=7,origin=(0.5, 0.5, 0.5))
print(gt_bbox_3d)
print(gt_bbox_3d.gravity_center,gt_bbox_3d.dims,gt_bbox_3d.yaw)
print(gt_bbox_3d.gravity_center.tolist(),gt_bbox_3d.dims.tolist(),gt_bbox_3d.yaw.tolist())

[ 3.23  1.59  8.55  2.37  1.63  1.48 -1.47]
LiDARInstance3DBoxes(
    tensor([[ 3.2300,  1.5900,  7.8100,  2.3700,  1.6300,  1.4800, -1.4700]]))
tensor([[3.2300, 1.5900, 8.5500]]) tensor([[2.3700, 1.6300, 1.4800]]) tensor([-1.4700])
[[3.2300000190734863, 1.590000033378601, 8.550000190734863]] [[2.369999885559082, 1.6299999952316284, 1.4800000190734863]] [-1.4700000286102295]


In [ ]:
############# NOTE THAT WE ONLY CHANGE poses w.r.t CAM0 and instances! NO other poses changed
# @TRANSFORMS.register_module()
# class ego_transform:
#     def __init__(self, trans='lidar2ego'):
#         self.T = trans
#     def transformation(self,results, Tr):
#         Tr = np.array(results['lidar_points'][self.T])
#         inv_Tr = np.linalg.inv(Tr)
#         results['gt_bboxes_3d'].rotate(Tr[:3,:3].T) # LidarInstance.rotate use right-side matmul
#         results['gt_bboxes_3d'].translate(Tr[:3,3])
#         results['lidar2cam'] = np.array(results['lidar2cam']) @ inv_Tr # @ pt_new
#         trans = results.get('trans_mat',np.identity(4))
#         results['trans_mat'] = Tr @ trans # @ pt_origin
#         return results

# @TRANSFORMS.register_module()
# class EgoTranslate:
#         new2old = np.identity(4)
#         new2old[:3,3] = t
#         old2new = np.identity(4)
#         old2new[:3,3] = -t
#         results['gt_bboxes_3d'].translate(-t)
#         results['lidar2cam'] = np.array(results['lidar2cam']) @ new2old # @ pt_new
#         trans = results.get('trans_mat',np.identity(4))
#         results['trans_mat'] = old2new @ trans # @ pt_origin
#         # return results